In [4]:
!python --version

Python 3.10.12


# Dependencies

In [5]:
!pip install roboflow ultralytics supervision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.1/83.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.2/922.2 kB 21.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 14.0 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.10.0 requires tensorflow==2.17.0, but you have tensorflow 2.17.1 which is incompatible.


In [6]:
import os
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt

In [7]:
from roboflow import Roboflow
from ultralytics import YOLO
from IPython.display import Image as IPyImage

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


# Importing dataset

In [8]:
HOME = os.getcwd()

In [9]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

/kaggle/working/datasets


In [10]:
rf = Roboflow(api_key="mO7qlobiaGbOPlDoDT4s")
project = rf.workspace("dimash").project("spacevela-3d-cv-1-class-fail")
version = project.version(16)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to SpaceVela-3D/CV-1-class-fail-16 in yolov11:: 100%|██████████| 2606/2606 [00:00<00:00, 7447.60it/s]


# Importing YOLO11-Seg model and Training

In [11]:
%cd {HOME}

/kaggle/working


In [12]:
print(dataset.location)

/kaggle/working/datasets/SpaceVela-3D/CV-1-class-fail-16


In [13]:
data_yaml_path = os.path.join(dataset.location, "data.yaml")

In [ ]:
# !yolo task=segment mode=train model=yolo11l-seg.pt data={dataset.location}/data.yaml epochs=120 imgsz=640 plots=True

In [ ]:
!yolo task=segment mode=train model=yolo11s-seg.pt data={data_yaml_path} epochs=30 imgsz=640 plots=True

100%|███████████████████████████████████████| 19.7M/19.7M [00:00<00:00, 127MB/s]
Ultralytics 8.3.82 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=segment, mode=train, model=yolo11s-seg.pt, data=/kaggle/working/datasets/SpaceVela-3D/CV-1-class-fail-16/data.yaml, epochs=300, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None

# Evaluating the Model

In [17]:
# Confusion Matrix
IPyImage(filename=f'{HOME}/runs/segment/train/confusion_matrix.png', width=600)

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/runs/segment/train/confusion_matrix.png'

In [ ]:
# Graphs for loss, precision, recall and mAP50
IPyImage(filename=f'{HOME}/runs/segment/train/results.png', width=600)

In [18]:
# Few instances of segmentation
IPyImage(filename=f'{HOME}/runs/segment/train/val_batch0_pred.jpg', width=800)

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/runs/segment/train/val_batch0_pred.jpg'

# Exporting the Model

In [ ]:
from google.colab import files

model_path = os.path.join(HOME, "runs", "segment", "train", "weights", "best.pt")
files.download(model_path)

In [ ]:
# model_path = os.path.join(HOME, "runs", "segment", "train")
# project.version(dataset.version).deploy(model_type="yolov11", model_path=model_path)

# References

In [ ]:
# How to use the model

'''
import cv2
import supervision as sv
from inference import get_model

model_id = project.id.split("/")[1] + "/" + dataset.version
model = get_model(model_id, "<ROBOFLOW_API_KEY>")

image = cv2.imread("<PATH_TO_IMAGE>")
result = model.infer(image)
detections = sv.Detections.from_inference(result)
'''

1. https://docs.ultralytics.com/tasks/segment
2. https://blog.roboflow.com/train-yolov11-instance-segmentation